In [11]:
import pandas as pd
import numpy as np
import torch
import pickle
from tqdm.auto import tqdm
from transformers import pipeline, AutoModel, AutoTokenizer

### Create train and test sets of tweets
Before feature extraction, we do the train-dev-test split, so that these datasets are constant across feature extraction methods. For binary classification, the positive examples come from the Russian troll tweets dataset. Negative examples are a combination of a sentiment dataset, and a dataset of tweets from Republican and Democratic politicians. (We want to make sure that there are negative examples that still have a "political" orientation, since our goal is to tell troll tweets from real tweets, rather than political from non-political.)

In [12]:
pos_examples = pd.read_csv("data/preprocessed-text/preprocessed-troll-tweets.csv")
neg_sentiment_examples = pd.read_csv("data/preprocessed-text/sentiment-preprocessed.csv",
                                    encoding="latin").rename(columns={"account_type":"account_category"})
neg_political_examples = pd.read_csv(
    "data/preprocessed-text/big-political-preprocessed.csv").rename(columns={
    "account_type":"account_category"})

In [13]:
len(pos_examples), len(neg_sentiment_examples), len(neg_political_examples)

(1970780, 1600498, 1243370)

In [14]:
neg_sentiment_examples.columns

Index(['content', 'account_category', 'troll'], dtype='object')

In [15]:
random_state = 229
combined = pd.concat([
    pos_examples,
    neg_sentiment_examples.sample(n=1000000, random_state=random_state),
    neg_political_examples.sample(n=1000000, random_state=random_state)
]).sample(frac=1, random_state=random_state).reset_index(drop=True)

In [16]:
len(combined)
combined.head(20)

,content,account_category,troll
0,.@BilgeEbiri really nails why many villains in...,RightTroll,True
1,"Clinton, Trump lead 2016 delegate race https:/...",NewsFeed,True
2,RT @the_intercept: How our reporter @JuanMThom...,LeftTroll,True
3,"Cruz, Colbert debate Reagan, gay marriage #en...",NewsFeed,True
4,'@420omnivore @leyalouisee May be this girl wi...,LeftTroll,True
5,More to Jamaica than 'anti-gay Gestapos': Man ...,NewsFeed,True
6,Darkness cannot drive out darkness; only light...,RightTroll,True
7,I'm thrilled to be here @ #CBCFALC2012 hosting...,NotTroll,False
8,"Nearly all men can stand adversity, but if you...",LeftTroll,True
9,‘Reprehensible’ fondling of 7-year-old girl by...,NewsFeed,True


In [17]:
metadata = {
    "troll": combined.groupby("account_category").count(), 
    "categories": combined.groupby("troll").count()
}
pickle.dump(metadata, open("data/preprocessed-text/combined-metadata.pickle", "wb+"))

In [18]:
train_cutoff, dev_cutoff = int(len(combined) * 0.7), int(len(combined) * 0.85)
train_tweets = combined.iloc[:train_cutoff,:]
dev_tweets = combined.iloc[train_cutoff:dev_cutoff,:]
test_tweets = combined.iloc[dev_cutoff:,:]

In [19]:
train_tweets.to_csv("data/preprocessed-text/train_tweets.csv", index=False)
dev_tweets.to_csv("data/preprocessed-text/dev_tweets.csv", index=False)
test_tweets.to_csv("data/preprocessed-text/test_tweets.csv", index=False)

### BERT features
Take these raw tweets for train, dev, and test sets and use pretrained BERT to create features.

In [21]:
train_tweets = pd.read_csv("data/preprocessed-text/train_tweets.csv",
                          dtype={'content':'string', 'account_category':'string', 'troll':'boolean'})
dev_tweets = pd.read_csv("data/preprocessed-text/dev_tweets.csv",
                          dtype={'content':'string', 'account_category':'string', 'troll':'boolean'})
test_tweets = pd.read_csv("data/preprocessed-text/test_tweets.csv",
                          dtype={'content':'string', 'account_category':'string', 'troll':'boolean'})

In [22]:
model_name = "distilbert-base-uncased"
model = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
# feature_extractor = pipeline('feature-extraction', model=model, tokenizer=tokenizer)

In [23]:
# Input: Pandas Series of tweets.
# Output: DataFrame of features where each column is a feature,
#         and each row is a tweet.
def bert_featurize(tweets, model, tokenizer):
    encoded = tweets.apply(lambda t: tokenizer.encode(t))
    max_len = np.max([len(t) for t in encoded.tolist()])
    padded = encoded.apply(lambda t: np.array(t + [0] * (max_len - len(t))))
    model_input = torch.tensor(np.vstack(padded.values))
    attention_mask = torch.tensor(np.where(model_input == 0, 0, 1))
    with torch.no_grad():
        output = model(model_input, attention_mask=attention_mask)
    return pd.DataFrame(output[0][:, 0, :].numpy())

In [36]:
# Input: Pandas DataFrame including "content" column for tweets.
# Output: Same DataFrame with BERT features added in new columns.
def bert_featurize_df(df, model, tokenizer, batch_size, outfile):
    for idx in tqdm(range(0, len(df), batch_size)):
        chunk = df.iloc[idx:idx + batch_size, :]
        bert_features = bert_featurize(chunk["content"], model, tokenizer)
        combined = chunk.reset_index(drop=True).join(bert_features.reset_index(drop=True))
        combined.to_csv(outfile, mode='a', index=False, header=False)
    

In [30]:
# simple test
bert_featurize_df(train_tweets[:200], model, tokenizer, batch_size=10, outfile="data/transformer-binary/tiny.csv")

Smaller datasets, taking a subsample of each of the splits.

In [ ]:
bert_featurize_df(train_tweets[:100000], model, tokenizer, batch_size=50, 
                  outfile="data/transformer-binary/bert_train_small.csv")

In [195]:
bert_featurize_df(dev_tweets[:15000], model, tokenizer, batch_size=50,
                 outfile="data/transformer-binary/dev_bert_small.csv")

In [ ]:
bert_featurize_df(test_tweets[:15000], model, tokenizer, batch_size=50,
                 outfile="data/transformer-binary/test_bert_small.csv")

Larger datasets, using the entirety of each split. (Takes a *long* time to run.)

In [37]:
bert_featurize_df(train_tweets.dropna()[1403050:], model, tokenizer, batch_size=25,
                 outfile="data/transformer-binary/train_bert_large.csv")

In [ ]:
dev_bert_large = bert_featurize_df(dev_tweets, model, tokenizer, batch_size=100)
dev_bert_large.to_csv("data/transformer-binary/dev_bert_large.csv")

In [ ]:
test_bert_large = bert_featurize_df(test_tweets, model, tokenizer, batch_size=100)
test_bert_large.to_csv("data/transformer-binary/test_bert_large.csv")

### "Bag of Words" Features

In [ ]:
from collections import defaultdict
import re

def create_vocab(tweets):
    word_counts = default_dict(int)
    # Use training data to construct vocabulary
    for row_idx in len(tweets):
        tweet = tweets['content'].iloc[row_idx] 
        words = re.sub(r'[^\w\s#]', '', tweet).lower().strip().split(" ")
        for word in words:
            word_counts[word] += 1

    # Filter out words that occur more than 0.75x the number of tweets, or less than 100 times.
    filtered = {k:v for k, v in word_counts.items() if v >= 100 and v < len(tweets) * 0.75}
    return {word: i for i, word in enumerate(filtered)}

In [ ]:
def tweet_to_arr(tweet, vocab):
    words = re.sub(r'[^\w\s#]', '', tweet).lower().strip().split(" ")
    idxs = [vocab[word] for word in words]
    arr = np.zeros((vocab,))
    arr[[idxs]] = 1
    return arr

In [ ]:
def bag_of_words_featurize(df, vocab):
    tweets = df.content
    arrs = tweets.apply(lambda tweet: tweet_to_arr(tweet, vocab))
    features = np.vstack(arrs.values)
    return df.reset_index(drop=True).join(pd.DataFrame(features).reset_index(drop=True))

In [ ]:
vocab = create_vocab(train_tweets)

In [35]:
len(pd.read_csv("data/transformer-binary/train_bert_large.csv"))

/Users/ben/.pyenv/versions/3.8.5/envs/data-analysis/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (1,2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


1403050